## In questo file andiamo ad individuare le 5 migliori feature combinado tutti gli algoritmi di feature selection

In [1]:
# Importiamo le librerie

from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from itertools import combinations
from os import listdir
from os.path import isfile, join

In [2]:
# Variabili globali

mypath = 'dataset'

# Mi vado a prendere i path di tutti i file nella cartella dataset

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Mi vado a generare tutte le possibili combinazioni di due elementi da un insieme di 5 elementi


comb = list(combinations(range(0,5), 2))

In [3]:
document = 'Friday-02-03-2018_TrafficForML_CICFlowMeter'

In [4]:
# Come prima cosa andiamo a leggere le feature dai vari file csv

anova = pd.read_csv('top_feature/'+document+"/anova.csv")
tree = pd.read_csv('top_feature/'+document+"/ExtraTree.csv")
svm = pd.read_csv('top_feature/'+document+"/SVM.csv")
rfe = pd.read_csv('top_feature/'+document+"/RFE.csv")


In [5]:
# Andiamo a normalizzare tutti gli score delle feature, in maniera tale da avere valori compresi tra 0 e 1

anova['F_Score'] = anova['F_Score']/anova['F_Score'].max()
tree['F_Score'] = tree['F_Score']/tree['F_Score'].max()
svm['F_Score'] = svm['F_Score']/svm['F_Score'].max()
rfe['F_Score'] = rfe['F_Score']/rfe['F_Score'].max()

In [6]:
display(anova)

,Input_Features,F_Score
0,Bwd Pkt Len Max,1.000000
1,RST Flag Cnt,0.837338
2,ECE Flag Cnt,0.837328
3,Bwd Pkt Len Mean,0.776309
4,Bwd Seg Size Avg,0.776309
5,Bwd Pkt Len Std,0.722059
6,Fwd Seg Size Min,0.700071
7,Init Bwd Win Byts,0.640212
8,Pkt Size Avg,0.618322
9,Pkt Len Max,0.561326


In [7]:
# Facciamo una concatenazione dei vari dataframe.
# Saranno utilizzati per prendere le feature che si ripetono di più e che hanno i punteggi maggiori

total_feature = pd.concat([anova,tree,svm,rfe],ignore_index=True)
total_feature_count = pd.concat([anova,tree,svm,rfe],ignore_index=True)

In [8]:
# Andiamo a eseguire un operazione di group by, con successivo count delle occorrenze dei vari campi

total_feature_count = total_feature_count.groupby(['Input_Features']).count().reset_index().sort_values(by='F_Score',ascending=False)

# Andiamo a rinominare la colonna F_Score, che adesso contiene i valori di count

total_feature_count.rename(columns = {'F_Score':'Conto'}, inplace = True)
print(total_feature_count)

       Input_Features  Conto
15         Fwd Pkts/s      3
5        ECE Flag Cnt      3
9         Flow Pkts/s      3
17  Init Bwd Win Byts      3
0        ACK Flag Cnt      2
2    Bwd Pkt Len Mean      2
22       RST Flag Cnt      2
20        Pkt Len Max      2
8        Flow IAT Min      2
18  Init Fwd Win Byts      2
1     Bwd Pkt Len Max      2
16   Fwd Seg Size Min      2
21       Pkt Size Avg      1
19       PSH Flag Cnt      1
12       Fwd IAT Mean      1
14   Fwd Pkt Len Mean      1
13        Fwd IAT Min      1
11        Fwd IAT Max      1
10     Fwd Header Len      1
7       Flow IAT Mean      1
6        FIN Flag Cnt      1
4    Bwd Seg Size Avg      1
3     Bwd Pkt Len Std      1
23       URG Flag Cnt      1


In [9]:
# Delle delle feature più frequenti mi interessano le prime dieci

most_frequent = total_feature_count[:10]

print(most_frequent)

       Input_Features  Conto
15         Fwd Pkts/s      3
5        ECE Flag Cnt      3
9         Flow Pkts/s      3
17  Init Bwd Win Byts      3
0        ACK Flag Cnt      2
2    Bwd Pkt Len Mean      2
22       RST Flag Cnt      2
20        Pkt Len Max      2
8        Flow IAT Min      2
18  Init Fwd Win Byts      2


In [10]:
# Ci serve ancora il parametro F_Scores, quindi creiamo un nuovo dataframe con queste righe.
# Abbiamo le dieci righe, però, sono ancora presenti le ridondanze che prima avevamo eliminato.

test = pd.DataFrame(data=[],columns=total_feature.columns)

for a in most_frequent['Input_Features']:
    test = pd.concat([test,total_feature[total_feature['Input_Features'] == a]])

print(test)

       Input_Features   F_Score
14         Fwd Pkts/s  0.435594
25         Fwd Pkts/s  0.827847
35         Fwd Pkts/s  1.000000
2        ECE Flag Cnt  0.837328
11       ECE Flag Cnt  0.782799
28       ECE Flag Cnt  0.744979
15        Flow Pkts/s  0.417705
24        Flow Pkts/s  0.843071
36        Flow Pkts/s  1.000000
7   Init Bwd Win Byts  0.640212
23  Init Bwd Win Byts  0.914249
30  Init Bwd Win Byts  1.000000
12       ACK Flag Cnt  0.591686
20       ACK Flag Cnt  1.000000
3    Bwd Pkt Len Mean  0.776309
37   Bwd Pkt Len Mean  1.000000
1        RST Flag Cnt  0.837338
10       RST Flag Cnt  1.000000
9         Pkt Len Max  0.561326
16        Pkt Len Max  0.367574
18       Flow IAT Min  0.261071
33       Flow IAT Min  1.000000
13  Init Fwd Win Byts  0.486912
31  Init Fwd Win Byts  1.000000


In [11]:
# Creo una variabile all'interno del quale vado a salvare il groupby del precedente dataframe, solo che in uesto caso
# andiamo a sommare i punteggi di F_Scores, invece di contare le occorrenze.

group_by = test.groupby(['Input_Features'])['F_Score'].sum().reset_index()

In [12]:
print(group_by)

      Input_Features   F_Score
0       ACK Flag Cnt  1.591686
1   Bwd Pkt Len Mean  1.776309
2       ECE Flag Cnt  2.365106
3       Flow IAT Min  1.261071
4        Flow Pkts/s  2.260776
5         Fwd Pkts/s  2.263441
6  Init Bwd Win Byts  2.554461
7  Init Fwd Win Byts  1.486912
8        Pkt Len Max  0.928900
9       RST Flag Cnt  1.837338


In [13]:
# Facciamo un ordinamento del dataframe precedente

print(group_by.sort_values(by='F_Score',ascending=False))

      Input_Features   F_Score
6  Init Bwd Win Byts  2.554461
2       ECE Flag Cnt  2.365106
5         Fwd Pkts/s  2.263441
4        Flow Pkts/s  2.260776
9       RST Flag Cnt  1.837338
1   Bwd Pkt Len Mean  1.776309
0       ACK Flag Cnt  1.591686
7  Init Fwd Win Byts  1.486912
3       Flow IAT Min  1.261071
8        Pkt Len Max  0.928900


In [14]:
# A questo punto, terminiamo l'operazione andando a moltiplicare la frequenza per la somma dei punteggi delle varie feature.

last_feature = pd.DataFrame(data=[],columns=group_by.columns)

for a,b in zip(most_frequent['Input_Features'],most_frequent['Conto']):
    last_feature.loc[len(last_feature)] =  {'Input_Features':a,'F_Score':group_by[group_by['Input_Features'] == a]['F_Score'].values[0]*b}
    

In [15]:
# Facciamo una stampa ordinata

print(last_feature.sort_values(by='F_Score',ascending=False))

      Input_Features   F_Score
3  Init Bwd Win Byts  7.663383
1       ECE Flag Cnt  7.095318
0         Fwd Pkts/s  6.790324
2        Flow Pkts/s  6.782329
6       RST Flag Cnt  3.674675
5   Bwd Pkt Len Mean  3.552618
4       ACK Flag Cnt  3.183373
9  Init Fwd Win Byts  2.973823
8       Flow IAT Min  2.522142
7        Pkt Len Max  1.857799


In [16]:
# A questo punto ci interessano solo le prime 5 feature, e tutte le loro possibili combinazioni di due elementi.

esito_finale = pd.DataFrame(data=[],columns=['Feature_1','Feature_2'])
top_index = []

# Qui andiamo a prendere gli indici delle prime 5 features

for a in range(0,5):
    top_index.append(last_feature.sort_values(by='F_Score',ascending=False).index[a])
    
# Andiamo a inserire tutte le possibili combinazioni nel dataframe

for b in comb:

    esito_finale.loc[len(esito_finale)] =  {
        'Feature_1': last_feature.loc[top_index[b[0]]]['Input_Features'],
        'Feature_2': last_feature.loc[top_index[b[1]]]['Input_Features']
        }

In [17]:
print(esito_finale)

           Feature_1     Feature_2
0  Init Bwd Win Byts  ECE Flag Cnt
1  Init Bwd Win Byts    Fwd Pkts/s
2  Init Bwd Win Byts   Flow Pkts/s
3  Init Bwd Win Byts  RST Flag Cnt
4       ECE Flag Cnt    Fwd Pkts/s
5       ECE Flag Cnt   Flow Pkts/s
6       ECE Flag Cnt  RST Flag Cnt
7         Fwd Pkts/s   Flow Pkts/s
8         Fwd Pkts/s  RST Flag Cnt
9        Flow Pkts/s  RST Flag Cnt


In [19]:
esito_finale.to_csv('top_feature/'+document+'/combinazioni.csv',index=False,header=False)